In [1]:
import fastai
import fastai.utils
from fastai.fautils import *

import pandas as pd

from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt

In [3]:
full_data_path = os.path.expanduser('~/data/state-farm/')
samp_data_path = os.path.expanduser('~/data/sample-state-farm/')
data_path = samp_data_path
batch_size = 64

tb_callback = keras.callbacks.TensorBoard(log_dir=data_path, batch_size=batch_size)
callbacks = [tb_callback, TQDMNotebookCallback()]

# overview

This notebook shows a lot of exploratory work with the State Farm kaggle challenge.  The biggest takeaways are:

1. Find the smallest sample size that produces consistent results
1. Start with very small models and quickly work up in complexity, till you're overfitting
1. Selecting the initial training rate, and adjusting it through training is really important
1. Get familiar with data augmentation, but remember you can't precompute your convolutional layers
1. Dropout is super important, but the value is dependent on your training set size, so you need to relearn it after you finish with the sample set

# create batches

In [4]:
t_batches = get_batches(data_path + 'train', batch_size=batch_size)
v_batches = get_batches(data_path + 'valid', batch_size=2*batch_size, shuffle=False)

# (
#     val_classes, trn_classes, 
#     val_labels, trn_labels, 
#     val_filenames, filenames,
#     test_filename
# ) = get_classes(data_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# linear model

In [5]:
def get_lin_model():

    # starting with BatchNormalization saves us from having to normalize our input manually
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [6]:
lm = get_lin_model()
lm.summary()

lm.fit_generator(
    t_batches, 
    steps_per_epoch=t_batches.samples//t_batches.batch_size,
    validation_data=v_batches, 
    validation_steps=v_batches.samples//v_batches.batch_size,
    epochs=1
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       896       
_________________________________________________________________
flatten_1 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1505290   
Total params: 1,506,186
Trainable params: 1,505,738
Non-trainable params: 448
_________________________________________________________________
Epoch 1/1
23/23 [==============================] - 13s - loss: 12.6318 - acc: 0.1651 - val_loss: 12.3120 - val_acc: 0.2109


In [7]:
np.round(lm.predict_generator(t_batches, t_batches.samples//t_batches.batch_size)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]], dtype=float32)

Model is entirely predicting 2 of the classes.  Not very useful.  Lower the learning rate and try again.

In [8]:
lm = get_lin_model()

lm.optimizer.lr.assign(1e-5)
lm.fit_generator(
    t_batches, 
    t_batches.samples//t_batches.batch_size,
    validation_data=v_batches, 
    validation_steps=v_batches.samples//v_batches.batch_size,
    epochs=2, 
)

Epoch 1/2
23/23 [==============================] - 12s - loss: 13.0348 - acc: 0.1474 - val_loss: 13.6715 - val_acc: 0.1473
Epoch 2/2
23/23 [==============================] - 9s - loss: 13.8679 - acc: 0.1380 - val_loss: 13.3523 - val_acc: 0.1708


In [9]:
lm.optimizer.lr.assign(1e-3)
lm.fit_generator(
    t_batches, 
    t_batches.samples//t_batches.batch_size,
    validation_data=v_batches, 
    validation_steps=v_batches.samples//v_batches.batch_size,
    epochs=4,
)

Epoch 1/4
23/23 [==============================] - 12s - loss: 13.4310 - acc: 0.1655 - val_loss: 13.6890 - val_acc: 0.1328
Epoch 2/4
23/23 [==============================] - 9s - loss: 13.4663 - acc: 0.1645 - val_loss: 13.9174 - val_acc: 0.1261
Epoch 3/4
23/23 [==============================] - 9s - loss: 13.5993 - acc: 0.1549 - val_loss: 14.4631 - val_acc: 0.1027
Epoch 4/4
23/23 [==============================] - 9s - loss: 13.9777 - acc: 0.1326 - val_loss: 14.4631 - val_acc: 0.1027


Validation accuracy of 0.43 -- almost too good to be true, and definitely better than chance.

Now lets make sure validation set is large enough to ensure a stable accuracy metric across runs, so we aren't making the wrong generalization about the results we're observing.

In [10]:
r_batches = get_batches(data_path+'valid', batch_size=2*batch_size)
val_res = [lm.evaluate_generator(r_batches, r_batches.samples//r_batches.batch_size) for i in range(10)]
np.round(val_res, 2)

Found 1000 images belonging to 10 classes.


array([[ 14.63,   0.09],
       [ 14.54,   0.1 ],
       [ 14.7 ,   0.09],
       [ 14.61,   0.09],
       [ 14.59,   0.09],
       [ 14.75,   0.08],
       [ 14.59,   0.09],
       [ 14.61,   0.09],
       [ 14.66,   0.09],
       [ 14.64,   0.09]])

Consisent results.  Accuracy increases larger than 2% can't be attributed to chance.

In [11]:
lm2 = get_lin_model()

rates = [
    (1e-5, 2),
    (1e-4, 4)
]

fastai.utils.fit_generator(lm2, t_batches, rates, val_batches=v_batches)

# lrsched = keras.callbacks.LearningRateScheduler(
#     fastai.utils.list_rate_schedule([
#         (1e-5, 2),
#         (1e-4, 4)
#     ],
#     output=True
#     )
# )

# lm2.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=6, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample,
#     callbacks=[lrsched]
# )

Learning rate: 1e-05
Epoch 1/6
23/23 [==============================] - 12s - loss: 2.4357 - acc: 0.1963 - val_loss: 5.0879 - val_acc: 0.1618
Epoch 2/6
23/23 [==============================] - 9s - loss: 1.6260 - acc: 0.4574 - val_loss: 3.3356 - val_acc: 0.2288
Learning rate: 0.0001
Epoch 3/6
23/23 [==============================] - 10s - loss: 1.0376 - acc: 0.6643 - val_loss: 3.0205 - val_acc: 0.4219
Epoch 4/6
23/23 [==============================] - 9s - loss: 0.4676 - acc: 0.8491 - val_loss: 2.4110 - val_acc: 0.5234
Epoch 5/6
23/23 [==============================] - 9s - loss: 0.1602 - acc: 0.9630 - val_loss: 2.3619 - val_acc: 0.5112
Epoch 6/6
23/23 [==============================] - 9s - loss: 0.0725 - acc: 0.9891 - val_loss: 1.8592 - val_acc: 0.5636


In [12]:
%run ../scripts/state-farm/linear-model.py

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 224, 224, 3)       896       
_________________________________________________________________
flatten_4 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1505290   
Total params: 1,506,186
Trainable params: 1,505,738
Non-trainable params: 448
_________________________________________________________________
Learning rate: 1e-05
Epoch 1/6
23/23 [==============================] - 12s - loss: 2.3990 - acc: 0.2174 - val_loss: 3.9647 - val_acc: 0.2288
Epoch 2/6
23/23 [==============================] - 9s - loss: 1.5655 - acc: 0.4946 - val_loss: 2.7194 - val_acc: 0.3739
Learning rate: 0.0001
Epoch 3/6
23/23 [=======

# regularized linear model

In [13]:
def get_reglin_model():

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax', kernel_regularizer=l2(0.01))
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [14]:
rlm = get_reglin_model()

rates = [
    (1e-5, 2),
    (1e-4, 4)
]

fastai.utils.fit_generator(rlm, t_batches, rates, val_batches=v_batches)

# rlm.optimizer.lr.set_value(1e-5)
# rlm.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

# rlm.optimizer.lr.set_value(1e-4)
# rlm.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=4, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

Learning rate: 1e-05
Epoch 1/6
23/23 [==============================] - 12s - loss: 2.6101 - acc: 0.1932 - val_loss: 9.8490 - val_acc: 0.0982
Epoch 2/6
23/23 [==============================] - 9s - loss: 1.8493 - acc: 0.4415 - val_loss: 4.6037 - val_acc: 0.1853
Learning rate: 0.0001
Epoch 3/6
23/23 [==============================] - 9s - loss: 1.3177 - acc: 0.6497 - val_loss: 5.9545 - val_acc: 0.3158
Epoch 4/6
23/23 [==============================] - 9s - loss: 0.6405 - acc: 0.8744 - val_loss: 2.8313 - val_acc: 0.4342
Epoch 5/6
23/23 [==============================] - 9s - loss: 0.4010 - acc: 0.9389 - val_loss: 2.3450 - val_acc: 0.5670
Epoch 6/6
23/23 [==============================] - 9s - loss: 0.3007 - acc: 0.9793 - val_loss: 2.2227 - val_acc: 0.4989


Single linear model with regularization is getting 45% accuracy.  Dipped at the end which means it's continuing to overfit.

# single dense layer

In [15]:
def get_fc_model():

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [16]:
fc = get_fc_model()
fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 224, 224, 3)       896       
_________________________________________________________________
flatten_6 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               15052900  
_________________________________________________________________
batch_normalization_7 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1010      
Total params: 15,055,206
Trainable params: 15,054,558
Non-trainable params: 648
_________________________________________________________________


In [17]:
rates = [
    (1e-5, 2),
    (1e-2, 5)
]

fastai.utils.fit_generator(fc, t_batches, rates, val_batches=v_batches)

# fc.optimizer.lr.set_value(1e-5)
# fc.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

# fc.optimizer.lr.set_value(0.01)
# fc.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

Learning rate: 1e-05
Epoch 1/7
23/23 [==============================] - 12s - loss: 1.8998 - acc: 0.3744 - val_loss: 6.3468 - val_acc: 0.2098
Epoch 2/7
23/23 [==============================] - 9s - loss: 0.8821 - acc: 0.7784 - val_loss: 4.0477 - val_acc: 0.2042
Learning rate: 0.01
Epoch 3/7
23/23 [==============================] - 9s - loss: 2.0273 - acc: 0.3525 - val_loss: 11.4310 - val_acc: 0.1094
Epoch 4/7
23/23 [==============================] - 9s - loss: 1.2235 - acc: 0.6345 - val_loss: 6.0620 - val_acc: 0.2232
Epoch 5/7
23/23 [==============================] - 9s - loss: 0.8292 - acc: 0.7774 - val_loss: 4.2570 - val_acc: 0.2746
Epoch 6/7
23/23 [==============================] - 9s - loss: 0.5732 - acc: 0.8469 - val_loss: 3.3024 - val_acc: 0.2946
Epoch 7/7
23/23 [==============================] - 9s - loss: 0.3595 - acc: 0.9193 - val_loss: 2.7566 - val_acc: 0.3203


Beginning to learn the training set, but failing at validation.

# simple conv layers

In [18]:
def get_conv_model(t_batches=t_batches, v_batches=v_batches, train=True):

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    if not train:
        return model
    
    rates = [
        (1e-4, 2),
        (1e-3, 4)
    ]

    fastai.utils.fit_generator(model, t_batches, rates, val_batches=v_batches, callbacks=callbacks)
    
#     model.optimizer.lr.set_value(1e-4)
#     h = model.fit_generator(
#         t_batches, 
#         t_batches.nb_sample, 
#         nb_epoch=2, 
#         validation_data=v_batches, 
#         nb_val_samples=v_batches.nb_sample
#     )

#     model.optimizer.lr.set_value(1e-3)
#     h = model.fit_generator(
#         t_batches, 
#         t_batches.nb_sample, 
#         nb_epoch=4, 
#         validation_data=v_batches, 
#         nb_val_samples=v_batches.nb_sample
#     )
    
    return model

In [19]:
cm = get_conv_model()
# cm.summary()

Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 21s - loss: 1.5129 - acc: 0.5525 - val_loss: 2.6520 - val_acc: 0.2667
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.3996 - acc: 0.9160 - val_loss: 1.8672 - val_acc: 0.3842
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.2459 - acc: 0.9390 - val_loss: 1.2706 - val_acc: 0.6021
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.1049 - acc: 0.9810 - val_loss: 1.5278 - val_acc: 0.5023
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0279 - acc: 0.9966 - val_loss: 1.9250 - val_acc: 0.3612
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0068 - acc: 0.9993 - val_loss: 1.9116 - val_acc: 0.4094


Very rapidly learning the training data and failing to generalize to the validation dataset.

# data augmentation

In [20]:
for width in [0, 0.05, 0.1, 0.15]:

    print('#########', width, '#########')
    gen_t = image.ImageDataGenerator(width_shift_range=width)
    batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

    model = get_conv_model(batches)

######### 0 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 19s - loss: 1.4985 - acc: 0.5455 - val_loss: 3.4949 - val_acc: 0.2385
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.4131 - acc: 0.9029 - val_loss: 1.7394 - val_acc: 0.3672
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.3944 - acc: 0.9001 - val_loss: 1.3300 - val_acc: 0.4977
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.1146 - acc: 0.9832 - val_loss: 1.9021 - val_acc: 0.2661
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0221 - acc: 1.0000 - val_loss: 1.8942 - val_acc: 0.3440
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0095 - acc: 1.0000 - val_loss: 2.3486 - val_acc: 0.1101
######### 0.05 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 22s - loss: 1.8815 - acc:

In [21]:
for height in [0, 0.05, 0.1, 0.15]:

    print('#########', height, '#########')
    gen_t = image.ImageDataGenerator(height_shift_range=height)
    batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

    model = get_conv_model(batches)

######### 0 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 19s - loss: 1.5502 - acc: 0.5367 - val_loss: 2.2480 - val_acc: 0.3062
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.4149 - acc: 0.9132 - val_loss: 1.8061 - val_acc: 0.3225
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.3653 - acc: 0.9093 - val_loss: 1.8317 - val_acc: 0.3440
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.1074 - acc: 0.9797 - val_loss: 1.3539 - val_acc: 0.6651
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0258 - acc: 0.9986 - val_loss: 1.7495 - val_acc: 0.3658
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0087 - acc: 1.0000 - val_loss: 1.8804 - val_acc: 0.3177
######### 0.05 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 22s - loss: 1.8352 - acc:

In [22]:
for shear in [0, 0.05, 0.1, 0.15]:

    print('#########', shear, '#########')
    gen_t = image.ImageDataGenerator(shear_range=shear)
    batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

    model = get_conv_model(batches)

######### 0 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 19s - loss: 1.5558 - acc: 0.5238 - val_loss: 2.9720 - val_acc: 0.2683
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.3835 - acc: 0.9207 - val_loss: 2.1068 - val_acc: 0.2351
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.4006 - acc: 0.8992 - val_loss: 1.4293 - val_acc: 0.4922
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.0893 - acc: 0.9898 - val_loss: 1.8346 - val_acc: 0.3028
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0251 - acc: 0.9993 - val_loss: 2.2243 - val_acc: 0.2362
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0086 - acc: 1.0000 - val_loss: 2.7619 - val_acc: 0.2122
######### 0.05 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 21s - loss: 1.6510 - acc:

In [23]:
for rot in [0, 10, 20, 30]:

    print('#########', rot, '#########')
    gen_t = image.ImageDataGenerator(rotation_range=rot)
    batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

    model = get_conv_model(batches)

######### 0 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 19s - loss: 1.5521 - acc: 0.5353 - val_loss: 3.1117 - val_acc: 0.2294
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.4239 - acc: 0.9224 - val_loss: 1.8095 - val_acc: 0.3383
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.3859 - acc: 0.9084 - val_loss: 1.4298 - val_acc: 0.4862
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.0999 - acc: 0.9859 - val_loss: 1.4383 - val_acc: 0.5493
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0285 - acc: 0.9986 - val_loss: 1.9163 - val_acc: 0.2936
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0117 - acc: 0.9993 - val_loss: 2.0671 - val_acc: 0.2557
######### 10 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 21s - loss: 1.8915 - acc: 0

In [24]:
for chan in [0, 10, 20, 30]:

    print('#########', chan, '#########')
    gen_t = image.ImageDataGenerator(channel_shift_range=chan)
    batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

    model = get_conv_model(batches)

######### 0 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 19s - loss: 1.6964 - acc: 0.4925 - val_loss: 2.3370 - val_acc: 0.2546
Epoch 2/6
23/23 [==============================] - 19s - loss: 0.5138 - acc: 0.8861 - val_loss: 1.5645 - val_acc: 0.4335
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 0.3788 - acc: 0.8974 - val_loss: 1.5985 - val_acc: 0.4656
Epoch 4/6
23/23 [==============================] - 19s - loss: 0.0920 - acc: 0.9876 - val_loss: 2.0048 - val_acc: 0.3177
Epoch 5/6
23/23 [==============================] - 19s - loss: 0.0213 - acc: 1.0000 - val_loss: 2.3410 - val_acc: 0.2615
Epoch 6/6
23/23 [==============================] - 19s - loss: 0.0079 - acc: 1.0000 - val_loss: 2.5653 - val_acc: 0.2294
######### 10 #########
Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 22s - loss: 1.6602 - acc: 0

# all together

In [ ]:
# gen_t = image.ImageDataGenerator(
#     width_shift_range=0.1,
#     height_shift_range=0.05,
#     shear_range=0.1,
#     rotation_range=15,
#     channel_shift_range=20
# )
gen_t = image.ImageDataGenerator(
    width_shift_range=0.05,
    height_shift_range=0.15,
    shear_range=0.15,
    rotation_range=15,
    channel_shift_range=30
)

batches = get_batches(data_path + 'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Learning rate: 0.0001
Epoch 1/6
23/23 [==============================] - 22s - loss: 2.5926 - acc: 0.2052 - val_loss: 3.2549 - val_acc: 0.1628
Epoch 2/6
23/23 [==============================] - 19s - loss: 2.0610 - acc: 0.3122 - val_loss: 2.3277 - val_acc: 0.2374
Learning rate: 0.001
Epoch 3/6
23/23 [==============================] - 19s - loss: 2.0682 - acc: 0.3254 - val_loss: 2.3664 - val_acc: 0.2718
Epoch 4/6
23/23 [==============================] - 19s - loss: 1.6780 - acc: 0.4244 - val_loss: 1.5182 - val_acc: 0.5011
Epoch 5/6
23/23 [==============================] - 19s - loss: 1.4655 - acc: 0.5029 - val_loss: 1.5097 - val_acc: 0.4381
Epoch 6/6
23/23 [==============================] - 19s - loss: 1.3549 - acc: 0.5413 - val_loss: 1.4541 - val_acc: 0.5356


In [ ]:
rates = [
    (1e-4, 12),
    (1e-5, 18),
]

fastai.utils.fit_generator(model, batches, rates, val_batches=v_batches, callbacks=callbacks)

# model.optimizer.lr.set_value(0.0001)
# model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=25, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)

Learning rate: 0.0001
Epoch 1/30
23/23 [==============================] - 22s - loss: 1.2506 - acc: 0.5964 - val_loss: 1.4146 - val_acc: 0.5780
Epoch 2/30
23/23 [==============================] - 19s - loss: 1.1565 - acc: 0.6268 - val_loss: 1.3710 - val_acc: 0.5917
Epoch 3/30
23/23 [==============================] - 19s - loss: 1.1129 - acc: 0.6400 - val_loss: 1.3345 - val_acc: 0.6101
Epoch 4/30
23/23 [==============================] - 19s - loss: 1.1344 - acc: 0.6382 - val_loss: 1.2859 - val_acc: 0.6330
Epoch 5/30
23/23 [==============================] - 20s - loss: 1.0571 - acc: 0.6671 - val_loss: 1.2316 - val_acc: 0.6606
Epoch 6/30
23/23 [==============================] - 19s - loss: 1.0520 - acc: 0.6577 - val_loss: 1.1992 - val_acc: 0.6881
Epoch 7/30
23/23 [==============================] - 19s - loss: 0.9583 - acc: 0.6980 - val_loss: 1.2256 - val_acc: 0.6170
Epoch 8/30
23/23 [==============================] - 19s - loss: 0.9466 - acc: 0.6999 - val_loss: 1.1775 - val_acc: 0.5906
Ep

In [ ]:
vf_batches = get_batches(full_data_path + 'valid', batch_size=2*batch_size, shuffle=False)
out = model.evaluate_generator(vf_batches, vf_batches.samples//vf_batches.batch_size)
out

In [ ]:
model.summary()

In [ ]:
model.save(data_path + 'state-farm-cnn.h5')

# reload model

# full training plot

In [ ]:
# history = []

# gen_t = image.ImageDataGenerator(
#     width_shift_range=0.1,
#     height_shift_range=0.05,
#     shear_range=0.1,
#     rotation_range=15,
#     channel_shift_range=20
# )
# batches = get_batches(data_path + 'train', gen_t, batch_size=batch_size)

# model = get_conv_model(train=False)
# model.optimizer.lr.set_value(1e-4)
# h = model.fit_generator(
#     batches, 
#     batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )
# history.append(h)

# model.optimizer.lr.set_value(1e-3)
# h = model.fit_generator(
#     batches, 
#     batches.nb_sample, 
#     nb_epoch=4, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )
# history.append(h)

In [ ]:
# model.optimizer.lr.set_value(0.0001)
# h = model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# history.append(h)
# h = model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=25, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# history.append(h)

In [ ]:
# acc = []
# val_acc = []
# for h in history:
#     acc += h.history['acc']
#     val_acc += h.history['val_acc']
    
# plt.plot(acc)
# plt.plot(val_acc)

# submit

Compute test set output and actually submit.

In [ ]:
from fastai import kaggle

def submission_df(preds, test_batches, classes):
    # construct dataframe of the submission
    index = pd.Series(
        [f.split('/')[-1] for f in test_batches.filenames],
        name='img'
    )

    df = pd.DataFrame(
        preds[:test_batches.samples, :],
        index=index,
        columns=classes
    )

    return df.sort_index()

In [ ]:
test_batches = get_batches(
    full_data_path + 'test', shuffle=False, batch_size=batch_size * 2,
    class_mode=None)
train_batches = get_batches(
    full_data_path + 'train', shuffle=False, batch_size=batch_size,
    class_mode=None)
classes = sorted(train_batches.class_indices)

In [ ]:
model = keras.models.load_model(samp_data_path + 'state-farm-cnn.h5')

In [ ]:
# predict
preds = model.predict_generator(
    test_batches, 
    int(math.ceil(test_batches.samples / test_batches.batch_size))
)

In [ ]:
df = submission_df(preds, test_batches, classes)
df = df.clip(0.05, 0.95)
df.to_csv(full_data_path + 'submission.csv', index=True)

In [ ]:
import subprocess

cmd = [
    'kg',
    'submit',
    '-u', os.environ['KAGGLE_USERNAME'],
    '-p', os.environ['KAGGLE_PASSWORD'],
    '-c', 'state-farm-distracted-driver-detection',
    full_data_path + 'submission.csv'
]

subprocess.call(cmd)